## Import The ImageDataGenerator Library

In [1]:
from keras.preprocessing.image import ImageDataGenerator

# Image Data Agumentation

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

# Loading our data and performing data agumentation

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c1dae236c94e444092d79eb7f6061170 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RiSl-WHZSkZpRolx2hmwp9o2osm6RO3RRTZqnnMD6h6Z',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_2 = client_c1dae236c94e444092d79eb7f6061170.get_object(Bucket='imageclassification-donotdelete-pr-7wigdxumyxq2lq', Key='Rock_Classification_Dataset.rar')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile

zip_ref = zipfile.ZipFile(BytesIO(streaming_body_2.read()), 'r')
print(zip_ref.namelist())

BadZipFile: File is not a zip file

In [73]:
x_train=train_datagen.flow_from_directory(r'C:\Users\prasanthsnp\Downloads\Rock_Classification_Dataset\Rock_Classification_Dataset\train_set',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='categorical')

x_test=test_datagen.flow_from_directory(r'C:\Users\prasanthsnp\Downloads\Rock_Classification_Dataset\Rock_Classification_Dataset\train_set',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='categorical')

Found 700 images belonging to 5 classes.
Found 700 images belonging to 5 classes.


# Importing Necessary Libraries

In [11]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# Initializing the model

In [75]:
# Example for initialization
# model=Sequential()

# Adding CNN Layers

In [13]:
classifier = Sequential()

classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(32,(3,3),activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())

In [14]:
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=5,activation='softmax'))

classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

# Compiling the model

In [15]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Fitting the model

In [16]:
classifier.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train),
    epochs=20,validation_data=x_test,validation_steps=len(x_test)
)

NameError: name 'x_train' is not defined

# Save the Model

In [17]:
classifier.save('rock.h5')

model_json=classifier.to_json()

with open("model-bw.json","w") as json_file:
    json_file.write(model_json)

# Test the model

In [100]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model=load_model("rock.h5")

img=tensorflow.keras.utils.load_img(r"C:\Users\prasanthsnp\Downloads\Rock_Classification_Dataset\Rock_Classification_Dataset\test_set\marble\marble157.jpg",grayscale=False,target_size=(64,64))

x=tensorflow.keras.utils.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred=model.predict(x)
pred

1/1 [==============================] - 0s 77ms/step


array([[0., 0., 1., 0., 0.]], dtype=float32)

# predicting the output

In [101]:
index=['blue calcite','limestone','marble','olivine','red crystal']
result=str(index[int(np.where(pred[0]==1.0)[0][0])])
result

'marble'